# Task 4 

In [1]:
# KNN cosine
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
df = pd.read_csv('.csv /movies.csv')
movies_metadata = pd.read_csv('.csv /movies_metadata.csv',
                          usecols = ['id','imdb_id','title'])
links = pd.read_csv('.csv /links.csv')
ratings_small = pd.read_csv('.csv /ratings_small.csv')
links  = links.merge(ratings_small, on = 'movieId')

links.rename(columns = {'tmdbId':'id'}, inplace = True)
movies_metadata =movies_metadata.drop([29503,35587,19730])
movies_metadata['id'] =movies_metadata['id'].astype('int64')
movies_metadata =movies_metadata.merge(links,on='id')

In [10]:
df = movies_metadata
df = movies_metadata[['userId', 'movieId', 'rating', 'title']]
combine_movie_rating = df.dropna(axis = 0, subset = ['title'])

movie_ratingCount = (df.
                     groupby(by = ['title'])['rating'].
                     count().
                     reset_index().
                     rename(columns = {'rating': 'totalRatingCount'})
                     [['title', 'totalRatingCount']])

movie_ratingCount.head()
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')

rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,7,1,3.000,Toy Story,247
1,9,1,4.000,Toy Story,247
2,13,1,5.000,Toy Story,247
3,15,1,2.000,Toy Story,247
4,19,1,3.000,Toy Story,247


In [11]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_ratingCount['totalRatingCount'].describe())

count   8754.000
mean      11.406
std       24.549
min        1.000
25%        1.000
50%        3.000
75%       10.000
max      341.000
Name: totalRatingCount, dtype: float64


In [12]:
popularity_threshold = 50
rating_popular_movie= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
0,7,1,3.000,Toy Story,247
1,9,1,4.000,Toy Story,247
2,13,1,5.000,Toy Story,247
3,15,1,2.000,Toy Story,247
4,19,1,3.000,Toy Story,247


In [13]:
movie_features_df = rating_popular_movie.pivot_table(index='title', columns='userId', values='rating').fillna(0)
movie_features_df.head(10)

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey,0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
28 Days Later,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
300,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A Beautiful Mind,0.000,0.000,0.000,0.000,4.500,0.000,0.000,3.500,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000
A Bug's Life,0.000,0.000,0.000,0.000,3.500,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000
A Christmas Story,0.000,0.000,0.000,5.000,0.000,0.000,0.000,3.500,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,5.000
A Clockwork Orange,0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.500,3.000,0.000,0.000,0.000,0.000,0.000,3.000


In [14]:
# Collaborative based Recommendation System using KNN

from scipy.sparse import csr_matrix

# converting the matrix into array matrix
movie_features_df_matrix = csr_matrix(movie_features_df.values)


from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')

model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
rating_popular_movie['rating']=rating_popular_movie['rating'].astype('int')
label_encoder= LabelEncoder()
rating_popular_movie['title']=label_encoder.fit_transform(rating_popular_movie['title'])
X,y=rating_popular_movie[['title']],rating_popular_movie['rating']
leaf_size = list(range(1,50))
n_neighbors = list(range(0,10))
p=[1,2,6,10]
#convert to dictionary
knn=KNeighborsClassifier(n_neighbors=10)
nca_pipe=Pipeline([('knn',knn)])
nca_pipe.fit(X,y)
#print(nca_pipe.score(X_test,y_test))
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
clf = GridSearchCV(knn, hyperparameters, cv=10)
best_model = clf.fit(X_train,y_train)

/var/folders/vs/t6hclhb524l53jbbc2n222640000gn/T/ipykernel_94299/2593618365.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_popular_movie['rating']=rating_popular_movie['rating'].astype('int')
/var/folders/vs/t6hclhb524l53jbbc2n222640000gn/T/ipykernel_94299/2593618365.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_popular_movie['title']=label_encoder.fit_transform(rating_popular_movie['title'])


In [ ]:
from sklearn.metrics import accuracy_score
#Predict testing set
y_pred = best_model.predict(X_test)
#Check performance using accuracy
print(accuracy_score(y_test, y_pred))

In [ ]:
print(movie_features_df_matrix)

In [ ]:
def recommend_movie(movie_name):
    
    distances, indices = model_knn.kneighbors(movie_features_df.loc[movie_name,:].values.reshape(1, -1), n_neighbors = 6)
    
    for i in range(0, len(distances.flatten())):
        if i == 0:
            #print('Recommendations for {0}:\n'.format(movie_features_df.index[indices.flatten()[0]]))
            print('Recommendations for {0}:\n'.format(movie_name))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

In [ ]:
recommend_movie('Avatar')